In [ ]:
%matplotlib notebook
%matplotlib inline

import trendet
import investpy
import yfinance as yf
import mplfinance as fplt

import numpy as np
import pandas as pd
import seaborn as sns

import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from datetime import datetime
from subprocess import check_output

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Leitura da da base de dados API
cardano = yf.download("ADA-USD", start="2018-01-01", end="2021-10-14", order= 'ascending', interval='1d')
cardano.head(5)

> ### 3. Processamento/Tratamento de Dados

In [ ]:
#Percentual de duplicadas
print('-Verificando duplicadas....: %.2f%%' % cardano.index.duplicated().sum())
# Quantidade de Linhas
print('-Numero de linhas ldap.....:', cardano.shape[0]) 
# Quantidade de Colunas
print('-Numero de colunas ldap....:', cardano.shape[1])
#Linhas e colunas
print('-Linhas e Colunas..........:', cardano.shape ) 
#Identificar valores null no dataframe
print('-Valores NULL..............: %.2f%% ' % cardano.isnull().sum().sum())


In [ ]:
v_mercado = pd.read_csv("v_mercado.csv", delimiter=';', header=0, parse_dates=[0],
                        index_col=0)
v_mercado.index = pd.to_datetime(v_mercado.index)

v_mercado = v_mercado.sort_values(by='Date')  # Ordenar o dados pela Data
v_mercado.head(2)

In [ ]:
#Percentual de duplicadas
print('-Verificando duplicadas....: %.2f%%' % v_mercado.index.duplicated().sum())
# Quantidade de Linhas
print('-Numero de linhas ldap.....:', v_mercado.shape[0]) 
# Quantidade de Colunas
print('-Numero de colunas ldap....:', v_mercado.shape[1])
#Linhas e colunas
print('-Linhas e Colunas..........:', v_mercado.shape ) 
#Identificar valores null no dataframe
print('-Valores NULL..............: %.2f%% ' % cardano.isnull().sum().sum())


In [ ]:
#Merge dos dados cardano e v_mercado
data = pd.merge(cardano, v_mercado, on="Date", how="left")
data.head(2)

In [ ]:
#Percentual de duplicadas
print('-Verificando duplicadas....: %.2f%%' % data.index.duplicated().sum())
# Quantidade de Linhas
print('-Numero de linhas ldap.....:', data.shape[0]) 
# Quantidade de Colunas
print('-Numero de colunas ldap....:', data.shape[1])
#Linhas e colunas
print('-Linhas e Colunas..........:', data.shape ) 
#Identificar valores null no dataframe
print('-Valores NULL..............: %.2f%% ' % data.isnull().sum().sum())


In [ ]:
data.describe()

In [ ]:
#Criação da coluna ano-mes
data['Mes'] = data.index.to_period('M')
data.head(2)

In [ ]:
plt.figure(figsize=(40,7))

ax = sns.boxplot(x="Mes", y="Volume", data=data)
plt.grid()

In [ ]:
plt.figure(figsize=(18,8))
sns.lineplot(data=data['Close'],linewidth = 1.5 , label = 'Fechamento (Close)')
plt.grid()
plt.show()

In [ ]:
data = data.loc['2020-10-01':'2021-12-30']
data.head(2)

In [ ]:
#Cópia do datagrame 
obv = data.copy()
obv.head(2)

In [ ]:
plt.figure(figsize=(20,7))

ax = sns.boxplot(x="Mes", y="Volume", data=data)
plt.grid()

In [ ]:
plt.figure(figsize=(18,8))
sns.lineplot(data=data['Close'],linewidth = 1.5 , label = 'Fechamento (Close)')
plt.grid()
plt.show()

In [ ]:
data[['Open','High','Low','Close','Adj Close']].plot.line(figsize=(18,8), linewidth=0.5);
plt.grid();

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(data.corr(),
            annot = True,
            fmt = '.2f',
            cmap='Blues')
plt.title('Correlação entre variáveis CARDANO')
plt.show()

> ### 4.2 - Média Móvel Simples (MMS)

In [ ]:
data['M14'] = data.Close.rolling(14, min_periods=14).mean()
data['M25'] = data.Close.rolling(25, min_periods=25).mean()
data.head(4)

In [ ]:
plt.figure(figsize=(18,8))
sns.lineplot(data=data['M14'],linewidth = 1.5 , label = 'Media - 14 dias')
sns.lineplot(data=data['Close'],linewidth = 1.5 , label = 'Fechamento (Close)')
sns.lineplot(data=data['M25'],linewidth = 1.5 , label = 'Media - 25 dias ')
plt.grid()
plt.show()

### Criação de indicação de compra e venda com sinal 

In [ ]:
data['sinal'] = np.where(data['M14']>data['M25'],1,0)
data.head(5)

In [ ]:
data['pos'] = data['sinal'].diff()
data.head(3)

In [ ]:
plt.figure(figsize = (20,8))
data['Close'].plot(color = 'k',linestyle='--',alpha=0.4, label = 'Fechamento (Close)')
data['M14'].plot(color = 'b', label = 'Média 14 dias')
data['M25'].plot(color = 'g', label = 'Média 25 dias')

plt.plot(data[data['pos']== 1].index,
         data['M14'][data['pos']== 1],
         '^', markersize = 12, color = 'g', label = 'Indicação - Compra')

plt.plot(data[data['pos']== -1].index,
         data['M25'][data['pos']== -1],
         'v', markersize = 12, color = 'r', label = 'Indicação - Venda')

plt.ylabel('Preço em Dólar')
plt.xlabel("Período")
plt.title('Gráfico com sinais de compra e venda')
plt.legend()
plt.grid()
plt.show()

> ### 4.3 - Média Móvel Convergente e Divergente (MACD) 

In [ ]:
#Função para criar as colunas e sinais
def get_macd(preco, mlento, mlongo, suave):
    exp1 = preco.ewm(span=mlento, adjust=False).mean()
    exp2 = preco.ewm(span=mlongo, adjust=False).mean()

    macd = pd.DataFrame(exp1 - exp2).rename(columns={'Close': 'macd'})

    sinal = pd.DataFrame(macd.ewm(span=suave, adjust=False).mean()).rename(
        columns={'macd': 'sinal'})

    hist = pd.DataFrame(macd['macd'] - sinal['sinal']
                        ).rename(columns={0: 'hist'})

    frames = [macd, sinal, hist]

    df = pd.concat(frames, join='inner', axis=1)

    return df

cardano_macd = get_macd(data['Close'], 26, 12, 6)
cardano_macd.tail()

In [ ]:
#Criação da função estrategia_macd 
def estrategia_macd(prices, data):
    preco_compra = []
    preco_venda = []
    macd_sinal = []
    sinal = 0

    for i in range(len(data)):
        if data['macd'][i] > data['sinal'][i]:
            if sinal != 1:
                preco_compra.append(prices[i])
                preco_venda.append(np.nan)
                sinal = 1
                macd_sinal.append(sinal)
            else:
                preco_compra.append(np.nan)
                preco_venda.append(np.nan)
                macd_sinal.append(0)
        elif data['macd'][i] < data['sinal'][i]:
            if sinal != -1:
                preco_compra.append(np.nan)
                preco_venda.append(prices[i])
                sinal = -1
                macd_sinal.append(sinal)
            else:
                preco_compra.append(np.nan)
                preco_venda.append(np.nan)
                macd_sinal.append(0)
        else:
            preco_compra.append(np.nan)
            preco_venda.append(np.nan)
            macd_sinal.append(0)

    return preco_compra, preco_venda, macd_sinal


preco_compra, preco_venda, macd_sinal = estrategia_macd(
    data['Close'], cardano_macd)

In [ ]:
plt.figure(figsize=(18, 6))

ax1 = plt.subplot2grid((8, 1), (0, 0), rowspan=5, colspan=1)
ax2 = plt.subplot2grid((8, 1), (5, 0), rowspan=3, colspan=1)

ax1.plot(data['Close'], color='skyblue',
         linewidth=2, label='Fechamento (Close)')
ax1.plot(data.index, preco_compra, marker='^', color='g', alpha=0.5,
         markersize=12, label='Sinal - Compra', linewidth=0)
ax1.plot(data.index, preco_venda, marker='v', color='r', alpha=0.5,
         markersize=12, label='Sinal - Venda', linewidth=0)

ax1.grid()
ax1.legend()

ax1.set_title('SINAL - CARDANO - MACD')
ax2.plot(cardano_macd['macd'], color='grey', linewidth=1.5, label='MACD')
ax2.plot(cardano_macd['sinal'], color='skyblue', linewidth=1.5, label='Sinal')

for i in range(len(cardano_macd)):
    if str(cardano_macd['hist'][i])[0] == '-':
        ax2.bar(cardano_macd.index[i], cardano_macd['hist'][i], color='#ef5350')
        
    else:
        ax2.bar(cardano_macd.index[i], cardano_macd['hist'][i], color='#26a69a')

plt.legend(loc='lower right')
plt.grid()
plt.show()


> ### 4.4 - Índice de Força Relativa (RSI)

In [ ]:
def RSI(data, window=10, adjust=False):
    delta = data['Close'].diff().dropna()
    perda = delta.copy()
    ganho = delta.copy()

    ganho[ganho < 0] = 0
    perda[perda > 0] = 0

    ganho_ewm = ganho.ewm(com=window-1, adjust=adjust).mean()
    perda_ewm = abs(perda.ewm(com=window-1, adjust=adjust).mean())

    RS = ganho_ewm / perda_ewm
    RSI = 100 - 100/(1 + RS)

    return RSI
rsi = RSI(data)

In [ ]:
spy = data[1:]
spy['RSI'] = rsi
spy['ponto_venda'] = np.where((rsi > 70) & (rsi.shift(1) <= 70), 1, 0)
spy['ponto_compra'] = np.where((rsi < 30) & (rsi.shift(1) >= 30), 1, 0)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 6))

ax[0].plot(spy['Close'][-500:], label='Fechamento (Close)')
ax[1].plot(rsi[-500:], label='Fechamento (Close)', c='purple')
ax[1].axhline(y=70, c='blue', label='RSI - 70%')
ax[1].axhline(y=30, c='orange', label='RSI - 30%')
ax[0].legend()
ax[1].legend(loc='lower right', fontsize=8)

ax[1].scatter(spy[spy['ponto_compra'] == 1][-5:].index,
              spy[spy['ponto_compra'] == 1]['RSI'][-5:],
              c='green',alpha=1)

ax[1].scatter(spy[spy['ponto_venda'] == 1][-16:].index,
              spy[spy['ponto_venda'] == 1]['RSI'][-16:],
              c='red',alpha=1);
plt.grid()

> ### 4.5 - Método Estocástico

In [ ]:
n = 8
n_highest_high = data["High"].rolling(n).max() # Leitura do valor máxima
n_lowest_low = data["Low"].rolling(n).min() # Leitura do valor mínima

In [ ]:
data["%K"] = (
    (data["Close"] - n_lowest_low) /
    (n_highest_high - n_lowest_low)
) * 100

data.head(2)

In [ ]:
data["%D"] = data['%K'].rolling(3).mean()
data.head(2)

In [ ]:
data.dropna(inplace=True)
data.head(2)

In [ ]:
data["Slow %K"] = data["%D"]
data["Slow %D"] = data["Slow %K"].rolling(3).mean()
data.head()

In [ ]:
data[["Slow %K", "Slow %D"]].plot(figsize=(18,7))
plt.axhline(y=20.0, color='black', linestyle='--', linewidth=1)
plt.axhline(y=80.0, color='black', linestyle='--', linewidth=1)
plt.ylim(0, 100.0);
plt.grid()

In [ ]:
def plot_stochastic(data):
    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        sharex=True,
        figsize=(18, 8),
        gridspec_kw={"height_ratios": [3, 1]})

    ax1.plot(data.index, data["Close"], label="Fechamento")
    ax1.grid()
    ax1.legend()

    ax2.plot(data.index, data[["Slow %K"]], label='Estocástico Lento')
    ax2.plot(data.index, data[["Slow %D"]], label='MMA(3)', linewidth=1)
    ax2.axhline(y=80, color='r', linestyle='--', linewidth=1)
    ax2.axhline(y=20, color='r', linestyle='--', linewidth=1)
    ax2.set_ylim(0, 100)
    ax2.legend()
    
plot_stochastic(data)

In [ ]:
def stochastic(data, k_window=8, mma_window=3):
    
    n_highest_high = data["High"].rolling(k_window).max()
    n_lowest_low = data["Low"].rolling(k_window).min()
    
    data["%K"] = (
        (data["Close"] - n_lowest_low) / 
        (n_highest_high - n_lowest_low)
    ) * 100
    data["%D"] = data['%K'].rolling(mma_window).mean()
    
    data["Slow %K"] = data["%D"]
    data["Slow %D"] = data["Slow %K"].rolling(mma_window).mean()
    
    return data 

plot_stochastic(data)

> ### 4.6 - Suporte e Resistência

In [ ]:
# SUPORTE RESISTÊNCIA
# Bibliotecas utilizadas para Suporte e Resistência
import matplotlib.dates as mpl_dates 
from mplfinance.original_flavor import candlestick_ohlc
plt.rcParams ['figure.figsize'] = [20, 8] 
plt.rc ('font', size = 14)

In [ ]:
data['Date'] = pd.to_datetime(data.index)
data['Date'] = data['Date'].apply(mpl_dates.date2num)
data = data.loc[:,['Date', 'Open', 'High', 'Low', 'Close']]

In [ ]:
def suporte(data, i):
    support = data['Low'][i] < data['Low'][i-1] and data['Low'][i] < data['Low'][i+1] \
        and data['Low'][i+1] < data['Low'][i+2] and data['Low'][i-1] < data['Low'][i-2]
    return support


def resistencia(data, i):
    resistance = data['High'][i] > data['High'][i-1] and data['High'][i] > data['High'][i+1] \
        and data['High'][i+1] > data['High'][i+2] and data['High'][i-1] > data['High'][i-2]
    return resistance

In [ ]:
levels = []
for i in range(2, data.shape[0]-2):
    if suporte(data, i):
        levels.append((i, data['Low'][i]))
    elif resistencia(data, i):
        levels.append((i, data['High'][i]))

In [ ]:
#função que traça o preço e os níveis-chave juntos.
def plot_all():
    fig, ax = plt.subplots()

    candlestick_ohlc(ax, data.values, width=0.6,
                     colorup='green', colordown='red', alpha=0.8)

    date_format = mpl_dates.DateFormatter('%d %b %Y')
    ax.xaxis.set_major_formatter(date_format)
    fig.autofmt_xdate()

    fig.tight_layout()

    for level in levels:
        plt.hlines(level[1], xmin=data['Date'][level[0]],
                   xmax=max(data['Date']), colors='blue', linestyle='-', alpha=0.4, color = 'b')
    fig.show()
  
plot_all()
plt.grid()

In [ ]:
s =  np.mean(data['High'] - data['Low'])

In [ ]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [ ]:
levels = []
for i in range(2, data.shape[0]-2):
    if suporte(data, i):
        l = data['Low'][i]

        if isFarFromLevel(l):
            levels.append((i, l))

    elif resistencia(data, i):
        l = data['High'][i]

        if isFarFromLevel(l):
            levels.append((i, l))
plot_all()
plt.grid()

> ### 4.7 - Bandas de Bollinger (BB) 

In [ ]:
periodo = 26 # Periodo de 20 dias
multiplicar = 2 # Multiplicação por 2 

In [ ]:
# multiplicador do desvio padrão
data['BandaSuperior'] = data['Close'].rolling(periodo).mean() + data['Close'].rolling(periodo).std() * multiplicar
data['BandaInferior'] = data['Close'].rolling(periodo).mean() - data['Close'].rolling(periodo).std() * multiplicar

#Média 20 dias 
data['Banda_Media'] = data['Close'].rolling(periodo).mean()

plt.rcParams['figure.figsize'] = [18, 6]
plt.rc('font', size=11)
plt.plot(data['Close'], label="Fechamento (Close)")

plt.plot(data['BandaSuperior'], linewidth=1,
         label="Banda de Bollinger Superior")
plt.plot(data['Banda_Media'], linewidth=1, label="Banda de Bollinger Média")
plt.plot(data['BandaInferior'], linewidth=1,
         label="Banda de Bollinger Inferior")
plt.grid()
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(18, 6))
ax = fig.add_subplot(111)

# Obtenha valores de índice para o eixo X para DataFrame
x_axis = data.index.get_level_values(0)

# Linha um Bollinger Band de 20 dias sombreado 
ax.fill_between(x_axis, data['BandaSuperior'], data['BandaInferior'],
                color='mediumturquoise', label="Nível de Banda Bollinger")

# Gráfico de ajuste de preço de fechamento e médias móveis
ax.plot(x_axis, data['Close'], color='b', lw=1, label="Fechamento (Close)")

ax.plot(x_axis, data['Banda_Media'], color='w', alpha=1, lw=2, label="Média de 20 Dias")

# Impressão da Imagem
ax.set_title('20 Dias (Bollinger Band)', fontsize=12)
ax.set_xlabel('Data')
ax.set_ylabel('Fechamento (Close) preço em USD', fontsize=12)
plt.grid()
ax.legend()
plt.show()

> ### 4.8 - Tópics Trend – Tendência de Alta e Baixa

In [ ]:
# Leitura da da base de dados API com dados limpos para impressão da Tópics Trends
trend = yf.download("ADA-USD", start="2019-01-1", end="2021-10-5", order= 'ascending', interval='1d')
trend = trend.loc['2020-10-01':'2021-10-14']
trend.head(2)

In [ ]:
tren = trend
# Codigo para o movimento de tendencia alta e baixa
res = trendet.identify_df_trends(df=tren, column='Close')

trend.reset_index(inplace=True)

plt.figure(figsize=(20, 7))
ax = sns.lineplot(x=tren.index, y=tren['Close'])
ax.set(xlabel='Data')

labels = tren['Up Trend'].dropna().unique().tolist()

for label in labels:
    sns.lineplot(x=tren[tren['Up Trend'] == label].index,
                 y=tren[tren['Up Trend'] == label]['Close'],
                 color='green')

    ax.axvspan(tren[tren['Up Trend'] == label].index[0],
               tren[tren['Up Trend'] == label].index[-1],
               alpha=0.2,
               color='green')

for label in labels:
    sns.lineplot(x=tren[tren['Up Trend'] == label].index,
                y=tren[tren['Up Trend'] == label]['Close'],
                color='white', linestyle='dashed')
    
    ax.axvspan(tren[tren['Up Trend'] == label].index[0],
               tren[tren['Up Trend'] == label].index[-1],
               alpha=0.2,
               color='green')

labels = trend['Down Trend'].dropna().unique().tolist()

for label in labels:
    sns.lineplot(x=tren[tren['Down Trend'] == label].index,
                 y=tren[tren['Down Trend'] == label]['Close'],
                 color='red')

    ax.axvspan(tren[tren['Down Trend'] == label].index[0],
               tren[tren['Down Trend'] == label].index[-1],
               alpha=0.2,
               color='red')
for label in labels:
    sns.lineplot(x=tren[tren['Down Trend'] == label].index,
                 y=tren[tren['Down Trend'] == label]['Close'],
                 color='white', linestyle='dashed')

    ax.axvspan(tren[tren['Down Trend'] == label].index[0],
               tren[tren['Down Trend'] == label].index[-1],
               alpha=0.2,
               color='red')

locs, _ = plt.xticks()
labels = []

for position in locs[1:-1]:
    labels.append(str(trend['Date'].loc[position])[:-9])

plt.xticks(locs[1:-1], labels)
plt.grid()
plt.show()

> ### 4.8 - Tópics Trend – Tendência de Alta e Baixa

In [ ]:
#Configuração para plotar o gráfico da Fibonnacci
plt.figure(figsize=(18,7))
plt.title('Retração Fibonnacci - CARDANO')
plt.plot(data.index, data['Close'])

#Calculate o valor maximo e  mininmo com o preço do fechamento
preco_maximo = data['Close'].max()
minimo_preco = data['Close'].min()

#Cacula a diferença entre o preço maximo e mínimo (subtração)
diferenca = preco_maximo - minimo_preco

#Impressão do preço em cada nível
primeiro_nivel = preco_maximo - diferenca * 0.236   
segundo_nivel = preco_maximo - diferenca * 0.382  
terceiro_nivel = preco_maximo - diferenca * 0.5     
quarto_nivel = preco_maximo - diferenca * 0.618  

#Impressão do preço em cada nível
print("Nível Percentage\t", "\t Preço ($)")
print("Preço Máximo....: 00.0%\t\t", "{0:.3f}".format(preco_maximo))
print("Primeiro Nível..: 23.6%\t\t", "{0:.3f}".format(primeiro_nivel))
print("Segundo Nível...: 38.2%\t\t", "{0:.3f}".format(segundo_nivel))
print("Terceiro Nível..: 50.0%\t\t", "{0:.3f}".format(terceiro_nivel))
print("Quarto Nível....: 61.8%\t\t", "{0:.3f}".format(quarto_nivel))
print("Preço Mínimo....: 100.0%\t", "{0:.3f}".format(minimo_preco))

plt.axhline(preco_maximo, linestyle='-', alpha=0.5, color = 'red')
plt.axhline(primeiro_nivel, linestyle='-', alpha=0.5, color = 'orange')
plt.axhline(segundo_nivel, linestyle='-', alpha=0.5, color = 'yellow')
plt.axhline(terceiro_nivel, linestyle='-', alpha=0.5, color = 'green')
plt.axhline(quarto_nivel, linestyle='-', alpha=0.5, color = 'blue')
plt.axhline(minimo_preco, linestyle='-', alpha=0.5, color = 'purple')
plt.xlabel('Date',fontsize=12)
#Impressão do gráfico
plt.ylabel('Fechamento (Close) preço em USD',fontsize=12)
plt.grid()
plt.show()

> ### 4.10 - On Balance Volume (OBV)

In [ ]:
obv['variacao'] = obv['Close'].diff()
obv = obv[1:] # remove first row once it does not have a variation
obv.head()

In [ ]:
obv['OBV Change'] = np.where(
    obv['variacao'] > 0,
    obv['Volume'],
    np.where(obv['variacao'] < 0, -obv['Volume'], 0))
obv['OBV'] = obv['OBV Change'].cumsum()
obv.head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    sharex=True, 
    figsize=(20,8), 
    gridspec_kw={'height_ratios': [3, 1]})

# índice de valor onde vamos traçar uma linha horizontal
index = obv.index.get_loc("2021-05-01")

# Plot do preço
ax1.plot(obv.index, obv['Close'], label='Fechamento CARDANO')
ax1.grid()
ax1.legend()
ax1.axhline(y=obv["Close"][index], color='red', linestyle='--')

# Plot OBV
ax2.plot(obv.index, obv['OBV'], label='On Balance Volume (OBV)', color="b")
ax2.axhline(y=obv["OBV"][index], color='red', linestyle='--')
ax2.plot(obv.index, obv['Cap_Mercado'], label='Capitão de Mercado', color="#033660")
ax2.grid()
ax2.legend();

> # Previsão de série temporal com o prophet

In [ ]:
# Biblioteca para Prophet
from fbprophet import Prophet

In [ ]:
data = yf.download("ADA-USD", start="2018-01-01", end="2021-10-14")
data.head(2)

In [ ]:
data.reset_index(drop=False, inplace=True)
data.head(2)

In [ ]:
data = data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis = 1)
data.head(2)

In [ ]:
# Select Date and Price
#data = data[["Date","Adj Close"]] 
# Rename the features: These names are required for the model fitting
data = data.rename(columns = {"Date":"ds","Close":"y"}) 
data.head()

In [ ]:
cardano=Prophet()
cardano.fit(data)
futuro=cardano.make_future_dataframe(periods=12, freq='M')
previsao=cardano.predict(futuro)

# future_air =  futuro
# forecast_air = previsão

In [ ]:
cardano.plot(previsao,xlabel='Data',
                             ylabel='CARDANO')

In [ ]:
cardano=Prophet(seasonality_mode='multiplicative')
cardano.fit(data)
futuro=cardano.make_future_dataframe(periods=12, freq='M')
previsao=cardano.predict(futuro)


In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig_air = cardano.plot(previsao)
a = add_changepoints_to_plot(fig_air.gca(), cardano, previsao)

In [ ]:
cardano = Prophet(changepoint_prior_scale=0.50,
                  seasonality_mode='multiplicative'
                  )
previsao = cardano.fit(data).predict(futuro)
fig_air = cardano.plot(previsao)
a = add_changepoints_to_plot(fig_air.gca(), cardano, previsao)

In [ ]:
#Componentes do modelo de plotagem
cardano.plot_components(previsao);

> # Previsões de Séries Temporais 5 DIAS

In [ ]:
data = yf.Ticker("ADA-USD")
data = data.history(start="2020-10-01", end="2021-10-14", interval="1d", prepost=bool,
                      actions=False, threads=True, auto_adjust=True, back_adjust=True, rounding=True)
data.head(2)


In [ ]:
# Para este modelo, vamos reduzir o periodo
data = data.loc['2021-01-01':'2021-10-1']

In [ ]:
#Removendo as colunas desnecessárias para o modelo
data = data.drop(['Open', 'High', 'Low', 'Volume'], axis = 1)
data.head(2)

In [ ]:
#Definir o mode para 55 semanas
semanas = 55
treino = data.iloc[:-semanas,0:1].copy()
teste = data.iloc[-semanas:,0:1].copy()

In [ ]:
# Plotar o gráfico de Treino e Teste
plt.figure(figsize=(18, 5))
plt.title('Valor do Fechamento - Close')
plt.plot(treino['Close'], color='b')
plt.plot(teste['Close'], color='orange')
plt.legend(['Treino', 'Teste'])
plt.xlabel('Data')
plt.ylabel('Valor')
plt.show()

A partir de agora só iremos usar a base de treino para fazer os estudos, a base de teste será somente usada para validar as previsões que iremos fazer.

Toda série temporal pode ser decomposta em 3 partes: tendência, sazonalidade e resíduo, que é o que resta após retirar da série as duas primeiras partes, abaixo a separação dessas partes:

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
#gráfico semanal com ciclos que se repetem a cada 55 semanas 
season = seasonal_decompose(treino, freq=55)
fig = season.plot();
fig.set_size_inches(16,8)

Claramente a série tem uma tendência de alta, com picos entre o final e inicio de anos e mínimas entre fevereiro e maio 

Porém é indicado utilizar testes estatísticos para confirmar se a séries é estacionária, vamos usar dois testes: o teste de Dickey Fuller e o teste KPSS.

Primeiramente vamos usar o teste de Dickey Fuller, usarei o valor P base de 5%, ou seja, caso o valor P esteja abaixo desses 5% significa que a série é estatisticamente estacionária.

Além disso, existe o Teste estatístico do modelo, onde estes valores podem ser comparados com os valores críticos de 1%, 5% e 10%, caso o teste estatístico esteja abaixo de algum valor critico escolhido a série será estacionária:

In [ ]:
#criando uma função para plotar o gráfico e mostrar o resultado do teste:
def checar_estacionariedade(y, lags_plots=48, figsize=(22,8)):
    "Use Series como parâmetro"
    
    # Criando plots do DF
    y = pd.Series(y)
    fig = plt.figure()

    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((3, 3), (1, 0))
    ax3 = plt.subplot2grid((3, 3), (1, 1))
    ax4 = plt.subplot2grid((3, 3), (2, 0), colspan=2)

    y.plot(ax=ax1, figsize=figsize, color='teal')
    ax1.set_title('Série de Preços')
    plot_acf(y, lags=lags_plots, zero=False, ax=ax2, color='teal');
    plot_pacf(y, lags=lags_plots, zero=False, ax=ax3, method='ols', color='teal');
    sns.distplot(y, bins=int(sqrt(len(y))), ax=ax4, color='teal')
    ax4.set_title('Distribuição dos Preços')

    plt.tight_layout()
    
    print('Resultados do teste de Dickey-Fuller:')
    adfinput = adfuller(y)
    adftest = pd.Series(adfinput[0:4], index=['Teste Statistico','Valor-P','Lags Usados','Números de Observações'])
    adftest = round(adftest,4)
    
    for key, value in adfinput[4].items():
        adftest["Valores Críticos (%s)"%key] = value.round(4)
        
    print(adftest)

In [ ]:
from math import sqrt
checar_estacionariedade(treino['Close'], lags_plots=60)

Neste caso o teste de Dickey Fuller indicou que a série não é estacionária (Valor P de 96% e o valor crítico de 5% é menor que o teste estatístico)

Agora vamos analisar a série com o teste KPSS, ao contrário do teste de Dickey Fuller, o teste KPSS já pressupõe que a série é estacionária e só não será se o valor P for inferior a 5% ou o teste estatístico for menor que algum valor crítico escolhido:

In [ ]:
checar_estacionariedade(treino['Close'].diff().diff().dropna(), lags_plots=60)

Confirmando o teste de Dickey Fuller, o teste KPSS também mostra que a série não é estacionária, pois o valor P está em 0% e o teste estatístico está acima de qualquer valor crítico.

# Criando o Modelo

Inicialmente o modelo será criado com base nos dados de treino, assim, os coeficientes para os 3 lags auto correlacionados serão gerados e servirão para testar a extrapolação de 5 dias consecutivos na base de testes.

Existem diversas maneiras de se fazer extrapolações para o futuro, as principais são:

 * Criar um modelo para cada dia a ser previsto e agregar tudo posteriormente,
 * Recursivo, prevendo o primeiro dia e usar essa previsão como base para prever o próximo dia.
 
Usaremos o último modelo neste exemplo, neste caso iremos prever os próximos 5 dias, comparar com dados reais, adicionar estes últimos novamente na base para fazer uma nova extrapolação, posteriormente iremos calcular o erro do modelo e analisar fatos relevantes da previsão.

In [ ]:
# Treinando o modelo
from statsmodels.tsa.arima_model import ARIMA
modelo = ARIMA(treino, order=(3, 1, 0)).fit()
pred_treino = modelo.predict()

In [ ]:
# Base de previsão com diferenciação
pred_treino[:5]

In [ ]:
# Voltando para a base de preços 
pred_treino[0] += treino.iloc[0,0]
pred_treino = np.cumsum(pred_treino)
pred_treino.head()

In [ ]:
treino['Pred Close'] = pred_treino
treino.dropna(inplace=True)
treino.head()

In [ ]:
# Plotando o gráfico
treino.plot(figsize=(18,6), 
            title='Fechamento (Close) - Real vs Previsto na base de Treino',
           color=['Teal','orangered'])
plt.ylabel('Preço')
plt.show()

In [ ]:
#Medindo o erro do treino pela métrica Root Mean Squared Error
from sklearn.metrics import mean_squared_error
erro = sqrt(mean_squared_error(treino['Close'], treino['Pred Close']))
print(f'O RMSE foi de R$ {round(erro,4)}/sc')

#### Obtendo os Coeficientes

Após treinar o modelo, podemos obter os coeficientes de multiplicação dos lags junto com o valor da constante, esses valores servirão como base para extrapolarmos as previsões para os próximos 5 dias.

In [ ]:
# Pegando os coeficientes de múltiplicação dos lags para usar na base de testes:
const, l1, l2, l3 = modelo.params

print(f'Valor da constante {round(const,4)}')
print(f'Coeficientes do Lag 1: {round(l1,4)}, Lag 2: {round(l2,4)} e Lag 3: {round(l3,4)}')

O próximo passo é criar uma função para fazer as previsões, usarei todas as bases com diferenciação, criarei a previsão para 5 períodos adiante e atualizarei os dados reais com mais 5 dias e assim sucessivamente até finalizarem os dados de teste.

In [ ]:
def prever_n_per(treino, teste, passos):
    
    # Adicionando a série com diferenciação para projetar previsões
    diff_treino = treino.diff().dropna().values
    hist = [x for x in diff_treino]

    diff_teste = teste.diff().dropna().values
    diff_teste = [x for x in diff_teste]

    # Adicionando primeira diferenciação no teste que ficou entre o treino e teste
    prim_diff = teste[0] - treino[-1]
    diff_teste.insert(0, prim_diff)
    
    prev = []

    for i in range(0,len(teste),passos):
        for j in range(passos):
    
            yhat = const + (l1 * hist[-1]) + (l2 * hist[-2]) + (l3 * hist[-3])
            prev.append(yhat)
            # usando a previsão atual como base para novas previsões
            hist.append(yhat)
        obs = diff_teste[i:i+passos]
        hist = list(np.append(hist,obs))
    return prev

In [ ]:
# Testando a função
steps = 5
prev = prever_n_per(treino['Close'], teste['Close'], passos=steps)
print('Previsões antes de reverter os preços: ')
print(prev[:5])

Do mesmo jeito que criamos a função para prever os preços com 5 dias de extrapolação, iremos criar a função abaixo para transformar os preços previstos para a base comparável com os preços reais da base de teste.

In [ ]:
def inverter_diff(treino, teste, previsão, passos):
    '''
    Usando o último dia da base de treino, após isso, usa o último valor previsto como base sucessivamente.
    
    A cada passo, volta a usar o primeiro valor da base de teste que se tornou real.
    '''
    treino = [x for x in treino]
    teste = [x for x in teste]
    prev_inv = []
    
    for i in range(0,len(teste), passos): 
        inverter = previsão[i] + treino[-1]
        prev_inv.append(inverter)
        
        for j in range(passos-1):
            inverter += previsão[i+j+1]
            prev_inv.append(inverter)
        treino = np.append(treino, teste[i:i+passos])
    return prev_inv

In [ ]:
# Adicionando a previsão na base de testes para comparar posteriormente
teste['Previsão 5D'] = inverter_diff(treino['Close'],
                                     teste['Close'],
                                     prev, passos=steps)

In [ ]:
def plotar_comparativo(orig, prev, passos, titulo='', figsize=(18,8)):
    plt.figure(figsize=figsize)
    
    orig = np.array(orig)
    prev = np.array(prev)
    plt.plot(orig, label='Real', color='teal')
    
    for i in range(0,len(orig), passos):
        eixo_x = [x for x in range(i,i+passos)]
        plt.plot(eixo_x, prev[i:i+passos], color='orangered')
    
    plt.title(titulo)
    plt.ylabel('Preços')
    plt.show()

In [ ]:
plotar_comparativo(teste['Close'], teste['Previsão 5D'], passos=steps, 
                   titulo='Comparativo do Fechamento - Valor Real x Previsto (Extrapolando 5 dias)')

In [ ]:
erro_teste = sqrt(mean_squared_error(teste['Close'], teste['Previsão 5D']))
print(f'O RMSE da base de testes foi de R$ {round(erro_teste,4)}/sc')

Avaliando o gráfico acima, nota-se que a principal característica desse tipo de previsão é que ele segue a tendência mais recente que está sendo negociada, porém ainda há espaço para melhoras, o modelo não é capaz de prever possíveis mudanças de tendências.

No próximo vai ser analisar o erro (previsto - real), pois neste tipo de análise pode-se obter informações para melhorar o modelo.

#### Analisando o Erro

Para analisar o erro criarei uma função semelhante às anteriores, irão ser analisados os valores médios de cada um dos 5 passos, normalmente a tendência é que o erro, neste caso o RMSE, aumente de acordo com os períodos que vão se expandindo devido às incertezas, veremos abaixo:

In [ ]:
def erro_n_passos(orig, prev, passos, plot=True):
    # Calculando o RMSE de cada linha
    erro = ((np.array(orig) - np.array(prev))**2)**(1/2)
    # Mudando o formato para ficar: (erros/exemplos, passos)
    erro = erro.reshape(int(len(erro)/passos), passos)
    
    # Fazendo a média do erro para cada passo
    media = erro.mean(axis=0)
    
    # Plotando o gráfico de erro
    if plot:
        indice = np.arange(0,passos)
        plt.figure(figsize=(12,4))
        plt.plot(media, 'o-', color='teal', markersize=8)
        plt.title('Erro Médio para Cada Passo', fontsize=15)
        plt.xlabel('Passos', fontsize=15)
        plt.ylabel('Erro', fontsize=15)
        for x,y in zip(indice, media):
            label = "{:.4f}".format(y)
            plt.annotate(label, (x, y), xytext=(x-0.1, y+.075), fontsize=12)
        plt.xticks(np.arange(passos), [x for x in range(1,passos+1)])
        plt.yticks([x for x in np.linspace(min(media)-0.2, max(media)+0.2, 10)])
        plt.grid(axis='y')
        plt.show()
    print(f'O RMSE para cada passo ficou assim: \n{media}')

In [ ]:
erro_passo = erro_n_passos(teste['Close'], teste['Previsão 5D'], steps)
erro_passo

In [ ]:
plt.figure(figsize=(20,5))

sns.lineplot(data=teste['Close'],linewidth = 1.5 , label = 'Fechamento (Close) ')
sns.lineplot(data=teste['Previsão 5D'],linewidth = 1.5 , label = 'Previsão 5 dias')


plt.show()

In [ ]:
#Criação média diária e média de 3 dias na previsão 
teste['Media'] = teste.Close.rolling(1, min_periods=1).mean()
teste['M3'] = teste["Previsão 5D"].rolling(3, min_periods=3).mean()

In [ ]:
#Criação da coluna SINAL com o cruzamento da Média diária com a Média de 5 dias

teste['sinal'] = np.where(teste['Media']>teste['M3'],1,0)
teste.head(2)

In [ ]:
teste['pos'] = teste['sinal'].diff()
teste.head(3)

In [ ]:
data1 = yf.Ticker("ADA-USD")
data1 = data1.history(start="2019-01-01", end="2021-09-30", interval="1d", prepost=bool,
                      actions=False, threads=True, auto_adjust=True, back_adjust=True, rounding=True)
data1.head(2)

data1 = data1.loc['2021-08-01':'2021-10-1',:]


In [ ]:
plt.figure(figsize = (20,8))
teste['Previsão 5D'].plot(color = 'y',linestyle='-',alpha=1, label = 'Previsão 5D')

data1['Open'].plot(color = 'k', label = 'Fechamento (Close)')


plt.plot(teste[teste['pos']== 1].index,
         teste['Previsão 5D'][teste['pos']== 1],
         '^', markersize = 11, color = 'g', label = 'Indicação - Compra')

plt.plot(teste[teste['pos']== -1].index,
         teste['Previsão 5D'][teste['pos']== -1],
         'v', markersize = 11, color = 'r', label = 'Indicação - Venda')

plt.plot(teste[teste['pos']== 1].index,
         teste['Previsão 5D'][teste['pos']== 1],
         'o', markerfacecolor="g", markersize = 21, color='r', alpha=0.1)

plt.plot(teste[teste['pos']== -1].index,
         teste['Previsão 5D'][teste['pos']== -1],
         'o', markerfacecolor="r", markersize = 21, color='r', alpha=0.1)

plt.ylabel('Valor')
plt.xlabel("Período")
plt.title('Predição com sinal de Compra e Venda')
plt.legend()
plt.grid()
plt.show()
